# Function Calling


In [ ]:
# !pip install pandas "mistralai>=0.1.2"

### Load API key

In [ ]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [ ]:
import pandas as pd

판다스를 이용해 데이터 프레임을 만듭니다.

In [ ]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [ ]:
df

#### function calling을 사용하지 않고 데이터 관련 질문에 답하는 방법
- 권장되는 방법은 아니지만 function calling을 사용하는 방식과 대조하기 위해 예시로 보여주고 있다고 이해하면 될 것 같습니다.

In [ ]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [ ]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

이 경우는 데이터 프레임을 직접 생성할 정도의 작은 규모였기 때문에 큰 문제가 되지 않습니다.

하지만 일반적으로 데이터를 다루면 적게는 수만 행, 많게는 수백만 행 이상을 처리해야 할 수도 있기 때문에 프롬프트로 데이터를 제공하는 것은 비현실적입니다.

이를 해결하기 위한 function calling에 대해 알아봅시다.

In [ ]:
response = mistral(prompt)
print(response)

## Step 1. 유저: tool과 query 명시하기

### Tools

- 모델이 호출하길 원하는 모든 tool을 정의합니다.

In [ ]:
import json

거래 id를 기준으로 지불 상태를 확인하는 함수를 정의합니다.

In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [ ]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

In [ ]:
type(status)

거래 id를 기준으로 지불 일자를 확인하는 함수를 정의합니다.

In [ ]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [ ]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

- 함수의 스펙을 JSON 형식으로 정의할 수 있습니다.
- 함수 자체 뿐만 아니라 여기에 입력되는 파라미터(argument)에 관한 설명 또한 포함되어 있습니다.

In [ ]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [ ]:
type(tool_payment_status)

In [ ]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [ ]:
type(tool_payment_status)

두 개의 함수를 묶어 줍니다.

In [ ]:
tools = [tool_payment_status, tool_payment_date]

In [ ]:
type(tools)

In [ ]:
tools

### functools

In [ ]:
import functools

`functools.partial`를 이용하여 위 데이터 프레임에 대한 함수로 고정해 줍니다.

In [ ]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [ ]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

In [ ]:
tools

### User query

- 예시: “What’s the status of my transaction?”

In [ ]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

## Step 2. Model: Generate function arguments 

채팅 기록에 담긴 유저의 요청 사항에 따라 필요한 tool을 모델이 스스로(auto) 고르도록 합니다.

In [ ]:
from mistralai.client import MistralClient

model = "mistral-large-latest"

client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"), endpoint=os.getenv("DLAI_MISTRAL_API_ENDPOINT"))

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

In [ ]:
response.choices[0].message.content

### 채팅 기록 저장하기
assistant의 응답을 채팅 기록에 추가합니다. (거래 ID가 필요하다고 응답합니다.)

그리고 이에 대한 유저의 추가 질문을 기록합니다. (거래 ID 답변)

In [ ]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

이를 바탕으로 모델의 응답을 이어서 생성합니다.

In [ ]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [ ]:
response

In [ ]:
response.choices[0].message

이번에는 'content'는 비어 있고, 'aurgments'에 값이 채워진 것을 볼 수 있습니다.

즉, 'name'에 명시된 함수에 전달할 인자를 생성한 것으로 이해할 수 있습니다.

In [ ]:
chat_history.append(response.choices[0].message)

- 주목해야 할 영역:
- `name='retrieve_payment_status'`
- `arguments='{"transaction_id": "T1001"}'`

## Step 3. 유저: tool 결과를 얻기 위해 function 실행하기

- 모델은 사용할 함수를 파악할 수 있지만, 이는 유저가 직접 실행해야 결과를 얻을 수 있습니다.

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

사용할 tool

In [ ]:
tool_function.name

tool에 입력될 인자

In [ ]:
tool_function.arguments

- function의 arugment는 string이 아닌 파이썬 dictionary여야 합니다.
- string을 dictionary로 만들기 위해 `json.loads()`를 이용할 수 있습니다.

In [ ]:
args = json.loads(tool_function.arguments)
print(args)

- 이전에 만들어 두었던 functools dictionary는 아래와 같습니다.

```Python
import functools
names_to_functions = {
    "retrieve_payment_status": 
      functools.partial(retrieve_payment_status, df=df),
    
    "retrieve_payment_date": 
      functools.partial(retrieve_payment_date, df=df),
}
```

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result

- 여기에서 function call의 결과는 chat message에 'tool' 사용 결과로 저장됩니다.

In [ ]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [ ]:
chat_history

## Step 4. 모델: 최종 응답 생성하기
- 모델은 이제 유저의 질문에 대한 최종 응답을 생성할 수 있습니다. 이때 'tool'로부터 획득한 정보를 활용합니다.


In [ ]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

### 직접 해보세요!
- 데이터와 관련된 다른 질문들을 직접 입력해보세요. 이를테면 "how much did I pay my recent order?" 와 같은 prompt를 사용할 수 있습니다.
